## System Initialization
To create a flake-polymer simulation start by stating your device type (cpu or gpu). Initialize your system by specifying the parameters (# chains, density, and flake structure if needed) and visualize.

In [1]:
import flowermd
import gsd
import gsd.hoomd
import hoomd
import matplotlib.pyplot as plt
import mbuild as mb
import numpy as np
import warnings
from cmeutils.visualize import FresnelGSD
from flowermd.base import Pack, Simulation, System, Molecule
from flowermd.base.forcefield import BaseHOOMDForcefield
from flowermd.library import KremerGrestBeadSpring, LJChain
from flowermd.library.forcefields import BeadSpring
from flowermd.utils import get_target_box_number_density
from mbuild.compound import Compound
from mbuild.lattice import Lattice
warnings.filterwarnings('ignore')
#device type
cpu = hoomd.device.CPU()
class Graphene(System):
    def __init__(
        self,
        x_repeat,
        y_repeat,
        n_layers,
        base_units=dict(),
        periodicity=(True, True, False),
    ):
        surface = mb.Compound(periodicity=periodicity)
        a = 3**.5
        lattice = Lattice(
            lattice_spacing=[a,a,a],
            lattice_vectors=  [[a,0,0],[a/2,3/2,0],[0,0,1]],
            lattice_points={"A": [[1/3,1/3,0], [2/3, 2/3, 0]]},
        ) 
        Flakium = Compound(name="F", element="F") # defines a carbon atom that will be used to populate lattice points
        layers = lattice.populate(
            compound_dict={"A": Flakium}, x=x_repeat, y=y_repeat, z=n_layers
        ) # populates the lattice using the previously defined carbon atom for every "A" site, repeated in all x,y, and z directions
        surface.add(layers) # adds populated carbon lattice layers to the 'surface' compound, which represents our graphene structure 
        surface.freud_generate_bonds("F", "F", dmin=0.9, dmax=1.1) # generates bonds depending on input distance range, scales with lattice
        surface_mol = Molecule(num_mols=1, compound=surface) # wraps into a Molecule object, creating "1" instance of this molecule

        super(Graphene, self).__init__(
            molecules=[surface_mol],
            base_units=base_units,
        )

    def _build_system(self):
        return self.all_molecules[0]
#system parameters
N_chains = 61
initial_dens = 0.001
final_dens = 0.3
kg_chain = LJChain(lengths=10,num_mols=N_chains)
sheet = Graphene(x_repeat=5, y_repeat=5, n_layers=1, periodicity=(False, False, False))
system = Pack(molecules=[Molecule(compound=sheet.all_molecules[0], num_mols=10), kg_chain], 
              density=initial_dens, packing_expand_factor = 6, seed=2)
system.visualize()

/Users/joselyngutierrez/miniconda3/envs/flowermd-dev/lib/python3.12/site-packages/gmso/core/element.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

## Creating Box and Packing Particles
To pack our system at the necessary denisty a manual box shrinking is required.

In [2]:
import unyt as u
target_box = get_target_box_number_density(density=final_dens*u.Unit("nm**-3"),n_beads=(500+(N_chains*10)))

## Force Field
To model non-attractive forces we set up a force field with the weeks chandler anderson (wca) potential

In [3]:
ff = BeadSpring(
    r_cut=2**(1/6),  
    beads={
        "A": dict(epsilon=1.0, sigma=1.0),  # chains
        "F": dict(epsilon=1.0, sigma=1.0),  # flakes
    },
    bonds={
        "F-F": dict(r0=1.0, k=1000.0),
        "A-A": dict(r0=1.0, k=1000.0),  # increased k to avoid chain collapse
    },
    angles={
        "A-A-A": dict(t0=2* np.pi / 3., k=100.0),   # moderate stiffness for chains
        "F-F-F": dict(t0=2 * np.pi / 3., k=5000),
    },
    dihedrals={
        "A-A-A-A": dict(phi0=0.0, k=0, d=-1, n=2), #need to turn this on later, messed up with straight chains
        "F-F-F-F": dict(phi0=0.0, k=500, d=-1, n=2),
    }
)

## Simulation
To run the simulation choose a small enough timestep, dt=0.0005 has worked best for these flake-polymer systems. 

In [ ]:
dt = 0.0005
gsd = f"{N_chains}_10mer10f_{dt}dt.gsd"
log = f"{N_chains}_10mer10f_{dt}dt.txt"
sim = Simulation(initial_state=system.hoomd_snapshot, forcefield=ff.hoomd_forces, device=cpu, dt = dt, gsd_write_freq=int(1000), log_file_name = log, gsd_file_name = gsd)
sim.run_update_volume(final_box_lengths=target_box, kT=6.0, n_steps=5e6,tau_kt=100*sim.dt,period=10,thermalize_particles=True)
sim.run_NVT(n_steps=1e6, kT=5, tau_kt=dt*100)
sim.flush_writers()
sim_visualizer = FresnelGSD(gsd_file=gsd, frame=-1, view_axis=(1, 1, 1))
sim_visualizer.view()

*Warning*: dihedral.harmonic: specified K <= 0


Initializing simulation state from a gsd.hoomd.Frame.
Step 1000 of 5000000; TPS: 1766.34; ETA: 47.2 minutes
Step 2000 of 5000000; TPS: 2788.59; ETA: 29.9 minutes
Step 3000 of 5000000; TPS: 3455.99; ETA: 24.1 minutes
Step 4000 of 5000000; TPS: 3924.75; ETA: 21.2 minutes
Step 5000 of 5000000; TPS: 4271.88; ETA: 19.5 minutes
Step 6000 of 5000000; TPS: 4538.17; ETA: 18.3 minutes
Step 7000 of 5000000; TPS: 4747.09; ETA: 17.5 minutes
Step 8000 of 5000000; TPS: 4918.55; ETA: 16.9 minutes
Step 9000 of 5000000; TPS: 5061.77; ETA: 16.4 minutes
Step 10000 of 5000000; TPS: 5181.08; ETA: 16.1 minutes
Step 11000 of 5000000; TPS: 5282.55; ETA: 15.7 minutes
Step 12000 of 5000000; TPS: 5367.02; ETA: 15.5 minutes
Step 13000 of 5000000; TPS: 5436.87; ETA: 15.3 minutes
Step 14000 of 5000000; TPS: 5506.91; ETA: 15.1 minutes
Step 15000 of 5000000; TPS: 5566.2; ETA: 14.9 minutes
Step 16000 of 5000000; TPS: 5619.49; ETA: 14.8 minutes
Step 17000 of 5000000; TPS: 5666.46; ETA: 14.7 minutes
Step 18000 of 5000000